# Replication of Nevo (2000)

- Paper: Nevo, Aviv, "A Practitioner's Guide to Estimation of Random‐Coefficients Logit Models of Demand," *Journal of Economics & Management Strategy*, 9 (2000), 513–548. [https://doi.org/10.1111/j.1430-9134.2000.00513.x](https://doi.org/10.1111/j.1430-9134.2000.00513.x)
- The Appendix of this paper is very useful as it provides details about the estimation steps. A copy can be found at [https://www.rasmusen.org/zg601/readings/Nevo.Ras_guide_appendix.pdf](https://www.rasmusen.org/zg601/readings/Nevo.Ras_guide_appendix.pdf)
- The original replication files are no longer maintained on Professor Nevo's website. Fortunately, thanks to Professor Eric Rasmusen, we can still access data files and updated MATLAB code (compatible with newer versions) at [https://rasmusen.org/zg604/lectures/blp/frontpage.htm](https://rasmusen.org/zg604/lectures/blp/frontpage.htm). This Jupyter notebook file builds upon those resources.
- The code could be cleaner by wrapping everything in a `class`. However, for easier debugging and explaining, I've kept the functions separate. If you'd like to see a class-based implementation, check out this reference: [https://github.com/daria-pus/BLP-Demand-Estimation/tree/master](https://github.com/daria-pus/BLP-Demand-Estimation/tree/master). Despite some differences in implementation, my results closely match those from it. Another option would be using `global`, but I generally avoid it since it can introduce subtle issues if not used carefully.
- I'd like to highlight that there's a well-established Python package called PyBLP, which can be installed via `conda install pyblp` or `pip install pyblp`. This package implements more advanced algorithms and comes with comprehensive documentation. For more details, visit its PyPI page: [https://pypi.org/project/pyblp/](https://pypi.org/project/pyblp/). That said, this replication still serves as excellent learning material for understanding the core principles of the BLP method.
- Last updated: Aug 14, 2025.

Five steps for the estimation (copy from my notes):
1. Prepare the data including draws from the distribution of $\nu$ and $D$;
2. For a given value of $\theta_2$ and $\delta$, compute the market shares implied by Equation (9);
3. For a given value of $\theta_2$, compute the vector $\delta$ that equates the market shares computed in Step 2 to the observed shares using Equation (10).
4. For a given $\theta_2$ (note that $\hat{\theta}_1$ can be analytically expressed by $\theta_2$), compute the error term using Equation (11) (as a function of the mean valuation computed in Step 3), interact it with the instruments, and compute the value of the objective function in Equation (7).
5. Search for the value of $\theta_2$ that minimizes the objective function computed in Step 4.

In [19]:
import sys
import numpy as np
import scipy

print("Python:", sys.version)
print("numpy:", np.__version__)
print("scipy:", scipy.__version__)

Python: 3.10.18 | packaged by conda-forge | (main, Jun  4 2025, 14:42:04) [MSC v.1943 64 bit (AMD64)]
numpy: 2.2.6
scipy: 1.15.2


In [ ]:
# Loading data

iv = scipy.io.loadmat('iv.mat')
ps2 = scipy.io.loadmat('ps2.mat')

iv = iv['iv'] # `iv` refers to the instrument matrix (Z). Its second to last columns are 20 instruments for the price variable. 
              # (The first column is the `id` label. Just ignore it.)
x2 = ps2['x2'] # `x2` is the matrix of variables with random coefficients (X2): constant term, price, sugar content, and mushy (whether
               # getting soggy in milk).
x1 = ps2['x1'] # `x1` is the matrix of variables entering the representative utility (X1): price and 24 brand fixed effects.
x1 = x1.toarray() # change this sparse matrix into a full ndarray
s_jt = ps2['s_jt'] # `s_jt` refers to market shares of the 24 brands in each market.
v = ps2['v'] # `v` refers to individual tastes: 94 rows refer to 94 markets, 80 columns refer to 20 individuals in each market 
             # with 4 random taste parameters per individual (corresponding to the four variables in x2, first 20 columns correspond 
             # to the first variable in x2, and so on).
demogr = ps2['demogr']  # `demogr` refers to demographics of individuals: 94 rows refer to 94 markets, 80 columns refer to 20 
                        # individuals in each market with four demographics variables: the log of income (Income), the log of income 
                        # squared, (Income Sq), Age, and Child (the author demeaned all these variables and might apply some other 
                        # transformation). the first 20 columns is the first demongraphics variable, the second 20 columns is the 
                        # second, and so on.

print(f'Shape of `iv`: {iv.shape}')
print(f'Shape of `x2`: {x2.shape}')
print(f'Shape of `x1`: {x1.shape}')
print(f'Shape of `s_jt`: {s_jt.shape}')
print(f'Shape of `v`: {v.shape}')
print(f'Shape of `demogr`: {demogr.shape}')

Shape of `iv`: (2256, 21)
Shape of `x2`: (2256, 4)
Shape of `x1`: (2256, 25)
Shape of `s_jt`: (2256, 1)
Shape of `v`: (94, 80)
Shape of `demogr`: (94, 80)


In [ ]:
# Preparation (1)

ns = 20
n_market = 94 # 47 cities × 2 quarters
n_brand = 24 # Therefore, we have 94 × 24 = 2256 observations in the dataset.
n_iv = 20 # number of instruments for price

## Create IV matrix
## Here creates a matrix IV from the instruments and the x's: 20 columns of instruments for price by default, and n_brand brand dummies.
Z = np.hstack([iv[:, 1:(n_iv+1)], x1[:, 1:(n_brand+1)]]) # iv[:, 1:(n_iv+1)]: the 20 IVs; x1[:, 1:(n_brand+1)]: 24 brand dummies
print(f'Shape of `Z`: {Z.shape}')

## Create market indicator
## `cdid` indicates the market each observation belongs to. `cdindex` records the index of the last observation in each market.
cdid = np.kron(np.arange(1, n_market+1), np.ones(n_brand))
cdindex = np.arange(n_brand-1, n_brand*n_market, n_brand)
print(f'Shape of `cdindex`: {cdindex.shape}')

## Set starting value for theta2
## There are 20 elements in theta2w and 13 parameters to be estimated (theta2): four characteristic parameters and nine interaction 
## parameters. The remaining seven elements are set to zero by assumption such as some interactions (e.g. sugar and income squared). 
## The first column refers to four characteristics parameters corresponding to v. The second to fifth columns (a 4 × 4 matrix) refers to
## the coefficient matrix of demographics, e.g., the second column refers to the marginal effects of the first demographics variable to 
## the four variables in X2.

## Prof. Nevo provided the following initial guess for theta2, but he didn't seem to explain how he derived it.
'''
theta2w = np.array([[0.3302, 5.4819, 0, 0.2037, 0],
                    [2.4526, 15.8935, -1.2000, 0, 2.6342],
                    [0.0163, -0.2506, 0, 0.0511, 0],
                    [0.2441, 1.2650, 0, -0.8091, 0]])
'''
## As an alternative, I've constructed the following matrix where elements simply indicate which parameters need to be estimated. 
## For the estimation process, my strategy involves randomly drawing initial guesses and using a loop to find a suitable starting 
## point for the algorithm.
theta2w = np.array([[1, 1, 0, 1, 0],
                    [1, 1, 1, 0, 1],
                    [1, 1, 0, 1, 0],
                    [1, 1, 0, 1, 0]])
print(f'Shape of `theta2w`: {theta2w.shape}')

## Identify non-zero elements in `theta2w`
## Here create a vector of the non-zero elements in `theta2w`, and the corresponding row and column indices. This facilitates passing 
## values to the functions later.
theti, thetj = np.nonzero(theta2w)
sort_idx = np.lexsort([theti, thetj])  # This step is crucial in Python. Unlike MATLAB, Python prioritizes row-major order by default. 
                                       # Without this step, `theta2`` would extract non-zero elements row by row (taking the first row's 
                                       # non-zeros first, and so on). But we want the coefficients of `v`` (the first column) grouped 
                                       # together. This step adjusts the iteration order to column-major instead.
theti = theti[sort_idx] 
thetj = thetj[sort_idx]
theta2 = theta2w[theti, thetj]

print(f'Shape of `theta2`: {theta2.shape}')

Shape of `Z`: (2256, 44)
Shape of `cdindex`: (94,)
Shape of `theta2w`: (4, 5)
Shape of `theta2`: (13,)


In [ ]:
# Preparation (2)

## Create weight matrix (Phi = Z'Z)
invPhi = np.linalg.inv(Z.T @ Z)
print(f'Shape of `invPhi`: {invPhi.shape}')

## Mean utility from a Logit model (for constructing the initial guess of the contraction mapping, see Step 3 in my notes)
temp = np.cumsum(s_jt, axis=0)
sum1 = temp[cdindex, :]
sum1[1:] = np.diff(sum1, axis=0)
outside_share = 1 - sum1
outside_share = np.repeat(outside_share, n_brand, axis=0) # compute the outside good market share by market
delta = np.log(s_jt) - np.log(outside_share) 

## Calculate the optimized theta1 given delta
mid = x1.T @ Z @ invPhi @ Z.T
theta1 = np.linalg.inv(mid @ x1) @ mid @ delta # Eq (14) in my notes
expmvalold = np.exp(x1 @ theta1) # Here uses x1 @ theta1 us the initial guess of delta. Later we will use the exponential 
                                 # form of the contraction mapping (Eq (13) in my notes).
theta2old = np.zeros(theta2.shape)

vfull = np.repeat(v, n_brand, axis=0)
dfull = np.repeat(demogr, n_brand, axis=0)

Shape of `invPhi`: (44, 44)


In [ ]:
# Define a function to computes the non-linear part of the utility (mu_ijt)

def mufunc(x2, theta2w, ns, vfull, dfull):
    '''
    This function computes the non-linear part of the utility (mu_ijt).
    '''
    n, K = x2.shape
    j = theta2w.shape[1] - 1
    mu = np.zeros([n, ns])

    for i in range(ns):
        v_i = vfull[:, i:K*ns:ns] # extract v for the i-th individual in each market
        d_i = dfull[:, i:j*ns:ns] # extract D (4×4 matrix) for the i-th individual in each market
        mu[:, i:i+1] = (x2 * v_i) @ theta2w[:, 0].reshape([-1, 1]) + x2 * (d_i @ theta2w[:, 1:].T) @ np.ones([K, 1]) 
        # The term (x2 * v_i) @ theta2w[:, 0].reshape([-1, 1]) computes the utility contribution from v through its effect on X2, 
        # and x2 * (d_i @ theta2w[:, 1:].T) @ np.ones([K, 1]) calculates the corresponding utility contribution from D through X2.

    return mu

In [ ]:
# Define a function to calculate the "individual" probabilities of choosing each brand (Eq (9) in my notes)

def ind_sh(expmval, expmu, ns, cdindex):
    '''
    This function calculates the "individual" probabilities of choosing each brand.
    '''
    eg = expmu * np.kron(np.ones([1, ns]), expmval) # `expmu` is the idiosyncratic component and `np.kron(np.ones([1, ns]), expmval)` 
                                                    # is the mean utility component.
    temp = np.cumsum(eg, axis=0)
    sum1 = temp[cdindex, :]
    sum1[1:, :] = np.diff(sum1, axis=0)

    denom1 = 1 / (1 + sum1)
    denom = np.repeat(denom1, n_brand, axis=0)

    return eg * denom

In [7]:
# Define a function to calculate the market share for each product (Eq (9) in my notes)

def mktsh(expmval, expmu, ns, cdindex):
    '''
    This function computes the market share for each product.
    '''
    ind_share = ind_sh(expmval, expmu, ns, cdindex)
    share = ind_share.sum(axis=1).reshape([-1, 1]) / ns
    
    return share

In [ ]:
# Define a function to compute the mean utility level using contraction mapping

def meanval(theta2, theta2old, theti, thetj, x2, s_jt, expmvalold, ns, vfull, dfull, cdindex):
    '''
    This function computes the mean utility level.
    '''

    if max(abs(theta2 - theta2old)) < 0.01: # when the distance is small, use a stricter tolerance value
        tol = 1e-9
        flag = 0
    else:
        tol = 1e-6
        flag = 1

    theta2w = scipy.sparse.coo_matrix((theta2, (theti, thetj))).toarray() # expand the `theta2` vector to a full matrix containing 
                                                                          # those zero elements.
    mu = mufunc(x2, theta2w, ns, vfull, dfull)
    expmu = np.exp(mu)

    norm = 1
    delta_iter = 0
    while norm>tol: # contraction mapping
        expmval = expmvalold * s_jt / mktsh(expmvalold, expmu, ns, cdindex)
        t = abs(expmval - expmvalold)
        norm = t.max()
        expmvalold = expmval
        delta_iter += 1

    if (flag==1) and (np.isnan(expmval).max()<1):
        expmvalold = expmval
        theta2old = theta2

    return np.log(expmval)

In [ ]:
# Define a function to computes the GMM objective function

def gmmobjg(theta2, theta2old, theti, thetj, x1, x2, Z, invPhi, s_jt, expmvalold, ns, vfull, dfull, cdindex):
    '''
    This function computes the GMM objective function. f is the objective, and df is the gradient.
    '''
    delta = meanval(theta2, theta2old, theti, thetj, x2, s_jt, expmvalold, ns, vfull, dfull, cdindex)

    if np.isnan(delta).max() == 1:
        return 1e+10 # If the derived `delta` vector contains any NaN values, the subsequent code will fail. In such cases, 
                     # the function should terminate early and return a sufficiently large value for the GMM objective function.

    theta1 = np.linalg.inv(x1.T @ Z @ invPhi @ Z.T @ x1) @ x1.T @ Z @ invPhi @ Z.T @ delta # theta1_hat (Eq (14) in my notes)
    gmmresid = delta - x1 @ theta1 # Eq (11) in my notes
    f = (gmmresid.T @ Z @ invPhi @ Z.T @ gmmresid)[0, 0]
    
    printed_line = f'GMM objective: {f:7.4f}'
    print(printed_line.ljust(50), end='\r')
    return f

In [ ]:
# Define a function to culculate the Jacobian

def jacob(expmval, theta2, theti, thetj, x2, demogr, v, vfull, dfull, cdindex, ns, n_brand, n_market):
    '''
    This function computes the Jacobian of the implicit function that defines the mean utility (Ddelta = - A^{-1} @ B). 
    Remember that we have 2256 delta_jt (24 brands × 94 markets) and 13 non-zero parameters in theta2w, thus the Jaconbian is a 2256 × 13 matrix. See Eq (17) in my notes.
    '''
    theta2w = scipy.sparse.coo_matrix((theta2, (theti, thetj))).toarray()
    expmu = np.exp(mufunc(x2, theta2w, ns, vfull, dfull))
    shares = ind_sh(expmval, expmu, ns, cdindex)

    n, K = x2.shape # n = 2256, k = 4
    J = theta2w.shape[1] - 1 # J = 5 - 1 = 4
    B = np.zeros([n, K*(J+1)]) # initialize matrix B: np.zeros([2256, 20])
    rel = np.where(theta2w.T.flatten()!=0)[0] # Contrtuct a vector indicating which columns in B correspond to those non-zero 
                                              # parameters in theta2w. Later we will drop these columns.

    ## Computing (partial share) / (partial sigma)
    for i in range(K):
        xv = (x2[:, i:(i+1)] @ np.ones([1, ns])) * np.repeat(v[:, ns*i:ns*(i+1)], n_brand, axis=0)
        temp = np.cumsum(xv * shares, axis=0)
        sum1 = temp[cdindex, :]
        sum1[1:, :] = np.diff(sum1, axis=0)
        B[:, i] = (shares * (xv - np.repeat(sum1, n_brand, axis=0))).mean(axis=1)

    ## Computing (partial share) / (partial pi) 
    for j in range(J):
        d = np.repeat(demogr[:, ns*j:ns*(j+1)], n_brand, axis=0)
        temp1 = np.zeros([n, K])
        for i in range(K):
            xd = x2[:, i:(i+1)] @ np.ones([1, ns]) * d
            temp = np.cumsum(xd * shares, axis=0)
            sum1 = temp[cdindex, :]
            sum1[1:, :] = np.diff(sum1, axis=0)
            temp1[:, i] = (shares * (xd - np.repeat(sum1, n_brand, axis=0))).mean(axis=1)
        B[:, K*(j+1):K*(j+2)] = temp1

    ## Computing (partial share) / (partial delta) 
    A = np.zeros([n, n_brand]) # initialize matrix A: np.zeros([2256, 24])
    for i in range(n_market):
        temp = shares[i*n_brand:(i+1)*n_brand, :]
        H1 = temp @ temp.T
        A[i*n_brand:(i+1)*n_brand, :] = (np.diag(temp.sum(axis=1)) - H1) / ns

    ## Deriving the Jacobian
    f = np.zeros([n, (theta2w!=0).sum()]) # initialize the Jacobian: np.zeros([2256, 13]). Only those non-zero parameters of 
                                          # theta2w enter the Jacobian, thus Jacobian is 2256 × 13.
    for i in range(n_market):
        f[i*n_brand:(i+1)*n_brand, :] = - np.linalg.inv(A[i*n_brand:(i+1)*n_brand, :]) @ B[i*n_brand:(i+1)*n_brand, rel]

    return f

In [11]:
# Define a function to calculate the VCov matrix of the estimates

def var_cov(theta2, expmval, theta2old, theti, thetj, x1, x2, Z, s_jt, demogr, v, invPhi, expmvalold, ns, vfull, dfull, 
            cdindex, n_brand, n_market):
    '''
    This function computes the VCov matrix of the estimates.
    '''
    delta = meanval(theta2, theta2old, theti, thetj, x2, s_jt, expmvalold, ns, vfull, dfull, cdindex)
    theta1 = np.linalg.inv(x1.T @ Z @ invPhi @ Z.T @ x1) @ x1.T @ Z @ invPhi @ Z.T @ delta # theta1_hat
    gmmresid = delta - x1 @ theta1

    n_Z = Z.shape[1]
    temp = jacob(expmval, theta2, theti, thetj, x2, demogr, v, vfull, dfull, cdindex, ns, n_brand, n_market)
    a = np.concatenate([x1, temp], axis=1).T @ Z
    IVres = Z * (gmmresid @ np.ones([1, n_Z]))
    b = IVres.T @ IVres
    vcov = np.linalg.inv(a @ invPhi @ a.T) @ a @ invPhi @ b @ invPhi @ a.T @ np.linalg.inv(a @ invPhi @ a.T)

    return vcov

My three-step strategy for choosing the initial guess:
1. Generate candidate initial guesses for the optimizer using samples drawn from a normal distribution. For each candidate, evaluate the objective function; only accept an initial guess if its function value is below a certain threshold (e.g., < 1000). This step ensures that further optimization starts from a reasonably promising region in the parameter space and avoids spending resources exploring unfavorable starting points.
2. Apply the Nelder-Mead algorithm to the selected initial guess, using a relatively loose convergence tolerance (e.g., xatol/fatol = 0.001) and a moderate maximum iteration limit (e.g., maxiter = 999, maxfex = 9999). This "coarse" optimization aims to rapidly descend from the initial function value (typically in the high range, such as around 1000) to a much lower region (e.g., around 50).
3. Take the output from Step 2 as a new starting point and re-apply the Nelder-Mead algorithm, now with a very strict convergence tolerance (e.g., xatol/fatol = 1e-5) and a much higher iteration/function evaluation limit (e.g., maxiter = 99999, maxfex = 999999). 

In [12]:
# Search the `theta2` that minimizes the GMM objective function (1): search an acceptable random initial guess

np.random.seed(123456789)

gmmobj_value = 9999
initial_iter = 0
while gmmobj_value > 1000:
    theta2 = np.random.normal(0, 1, (theta2w!=0).sum())
    gmmobj_value = gmmobjg(theta2, theta2old, theti, thetj, x1, x2, Z, invPhi, s_jt, expmvalold, ns, vfull, dfull, cdindex)
    initial_iter += 1

print(f'\n# of iteration for searching an acceptable random initial guess: {initial_iter}')

GMM objective: 525.9014                           
# of iteration for searching an acceptable random initial guess: 67


In [13]:
# Search the `theta2` that minimizes the GMM objective function (2): use the random initial guess to find a better initial guess

options = {'maxiter': 999,
           'maxfev': 9999,
           'disp': True,
           'fatol': 0.001,
           'xatol': 0.001}

result = scipy.optimize.minimize(fun=gmmobjg,
                                 x0=theta2,
                                 args=(theta2old, theti, thetj, x1, x2, Z, invPhi, s_jt, expmvalold, ns, vfull, dfull, cdindex),
                                 method='Nelder-Mead',
                                 options=options)
theta2 = result.x
print(f'\n# of iteration for finding a better initial guess: {result.nit}')

GMM objective: 34.4632                            
# of iteration for finding a better initial guess: 999


C:\Users\Chen\AppData\Local\Temp\ipykernel_10232\1700612197.py:9: RuntimeWarning: Maximum number of iterations has been exceeded.
  result = scipy.optimize.minimize(fun=gmmobjg,


In [14]:
# Search the `theta2` that minimizes the GMM objective function (3): final solution

options = {'maxiter': 99999,
           'maxfev': 999999,
           'disp': True,
           'fatol': 1e-5,
           'xatol': 1e-5}


result = scipy.optimize.minimize(fun=gmmobjg,
                                 x0=theta2,
                                 args=(theta2old, theti, thetj, x1, x2, Z, invPhi, s_jt, expmvalold, ns, vfull, dfull, cdindex),
                                 method='Nelder-Mead',
                                 options=options)
theta2 = result.x
theta2

Optimization terminated successfully.             
         Current function value: 4.561502
         Iterations: 7612
         Function evaluations: 10207


array([ 5.58099481e-01,  3.31253036e+00, -5.78373419e-03,  9.34141074e-02,
        2.29198907e+00,  5.88324482e+02, -3.84954821e-01,  7.48380461e-01,
       -3.01919820e+01,  1.28443293e+00,  5.22344609e-02, -1.35338713e+00,
        1.10546814e+01])

In [15]:
# Calculate the standard errors

expmval = np.exp(meanval(theta2, theta2old, theti, thetj, x2, s_jt, expmvalold, ns, vfull, dfull, cdindex))
vcov = var_cov(theta2, expmval, theta2old, theti, thetj, x1, x2, Z, s_jt, demogr, v, invPhi, expmvalold, ns, vfull, dfull, 
            cdindex, n_brand, n_market)
se = np.sqrt(np.diag(vcov)).reshape([-1, 1])
se.flatten()[25:]

array([1.62533453e-01, 1.34018630e+00, 1.35045661e-02, 1.85433475e-01,
       1.20856430e+00, 2.70440180e+02, 1.21458091e-01, 8.02103568e-01,
       1.41011847e+01, 6.31213371e-01, 2.59852224e-02, 6.67106871e-01,
       4.12256129e+00])

In [16]:
# Calculate theta1 and its standard errors

delta = meanval(theta2, theta2old, theti, thetj, x2, s_jt, expmvalold, ns, vfull, dfull, cdindex)
theta1 = np.linalg.inv(x1.T @ Z @ invPhi @ Z.T @ x1) @ x1.T @ Z @ invPhi @ Z.T @ delta
theta1.flatten(), se[:25].flatten()

(array([-6.27298551e+01, -2.50288270e+00,  2.32639274e+00, -2.10234644e-01,
        -1.20252140e+00,  3.48218207e+00,  1.38538765e+00, -4.99599787e-01,
        -9.05209301e-01,  1.01219613e+00, -6.70634634e-01,  1.11258144e+00,
        -2.25082358e+00, -1.53474840e-01,  1.28161495e+00,  1.72333881e+00,
         3.55564189e+00,  3.79785356e-02, -1.13713485e+00,  1.76086079e+00,
         1.60251179e+00,  2.34670803e+00, -8.45654370e-01,  7.32257111e-01,
        -6.83677936e-01]),
 array([14.8031814 ,  0.85887488,  0.70585882,  0.77164154,  0.50101823,
         0.60996283,  0.63633522,  0.76117077,  0.47064202,  0.64514303,
         0.56466774,  0.59275293,  0.78013487,  0.74968608,  0.63103308,
         0.63363178,  0.87583797,  0.57072236,  0.48092638,  0.66256224,
         0.48942523,  0.65685708,  0.49591904,  0.58773008,  0.59044642]))

In [ ]:
# Minimum-distance procedure 
## Constant term, sugar content, and mushiness are brand-specific attributes that remain invariant across markets. These variables 
## are consequently absorbed into the brand fixed effects, even when explicitly included in X1. The minimum-distance estimation 
## procedure recovers these parameters by regressing the estimated fixed effects on the observed brand characteristics. (See Section 3.5
## of Nevo's paper or Section 3.6 of my notes)

omega = np.linalg.inv(vcov[1:25, 1:25]) # covariance matrix of fixed effects, use it for the WLS
xmd = x2[:24, [0, 2, 3]] # constant term, suggar content, and mushy
ymd = theta1[1:25] # 24 fixed effect estimates

beta = np.linalg.inv(xmd.T @ omega @ xmd) @ xmd.T @ omega @ ymd # WLS
semd = np.sqrt(np.diag(np.linalg.inv(xmd.T @ omega @ xmd))).reshape([-1, 1]) # calculate s.e.
mcoef = np.vstack([beta[0], theta1[0], beta[1:]]) # add the coef. of the price variable
semcoef = np.vstack([semd[0], se[0], semd[1:]]) # add the s.e. of the price variable
mcoef, semcoef

(array([[ -2.00992485],
        [-62.72985511],
        [  0.11625705],
        [  0.4993708 ]]),
 array([[ 0.32699748],
        [14.8031814 ],
        [ 0.01603637],
        [ 0.19858104]]))

In [ ]:
# Replicate Table 1 of Nevo's paper
## Note that the derived results are different from Nevo's. This is because the algorithm finds a smaller value (4.6) of the GMM
##  objective function than Nevo's (14.9).

print(f"————————————————————————————————————————————————————————————————————————————————————————————————————————")
print(f"{'':^15}{'Mean':^15}{'Std. Dev.':^15}{'Interation with Demographic Variables':^60}")
print(f"                                                ————————————————————————————————————————————————————————")
print(f"{'Variable':^15}{'β':^15}{'σ':^15}{'Income':^15}{'IncomeSq':^15}{'Age':^15}{'Child':^15}")
print(f"————————————————————————————————————————————————————————————————————————————————————————————————————————")
print(f"{'Price':^15}{mcoef[1].item():^15.3f}{theta2[1].item():^15.3f}{theta2[5].item():^15.3f}{theta2[8].item():^15.3f}{'':^15}{theta2[12].item():^15.3f}")
print(f"{'':^15}{'('+str(round(semcoef[1].item(), 3))+')':^15}{'('+str(round(se[26].item(), 3))+')':^15}{'('+str(round(se[30].item(), 3))+')':^15}{'('+str(round(se[33].item(), 3))+')':^15}{'':^15}{'('+str(round(se[37].item(), 3))+')':^15}")
print(f"{'Constant':^15}{mcoef[0].item():^15.3f}{theta2[0].item():^15.3f}{theta2[4].item():^15.3f}{'':^15}{theta2[9].item():^15.3f}{'':^15}")
print(f"{'':^15}{'('+str(round(semcoef[0].item(), 3))+')':^15}{'('+str(round(se[25].item(), 3))+')':^15}{'('+str(round(se[29].item(), 3))+')':^15}{'':^15}{'('+str(round(se[34].item(), 3))+')':^15}{'':^15}")
print(f"{'Sugar':^15}{mcoef[2].item():^15.3f}{theta2[2].item():^15.3f}{theta2[6].item():^15.3f}{'':^15}{theta2[10].item():^15.3f}")
print(f"{'':^15}{'('+str(round(semcoef[2].item(), 3))+')':^15}{'('+str(round(se[27].item(), 3))+')':^15}{'('+str(round(se[31].item(), 3))+')':^15}{'':^15}{'('+str(round(se[35].item(), 3))+')':^15}{'':^15}")
print(f"{'Mushy':^15}{mcoef[3].item():^15.3f}{theta2[3].item():^15.3f}{theta2[7].item():^15.3f}{'':^15}{theta2[11].item():^15.3f}")
print(f"{'':^15}{'('+str(round(semcoef[3].item(), 3))+')':^15}{'('+str(round(se[28].item(), 3))+')':^15}{'('+str(round(se[32].item(), 3))+')':^15}{'':^15}{'('+str(round(se[36].item(), 3))+')':^15}{'':^15}")
print(f"————————————————————————————————————————————————————————————————————————————————————————————————————————")
print(f"{'GMM objective':^15}{'':^15}{'':^15}{result.fun.item():^15.3f}{'':^15}{'':^15}{'':^15}")
print(f"————————————————————————————————————————————————————————————————————————————————————————————————————————")

————————————————————————————————————————————————————————————————————————————————————————————————————————
                    Mean         Std. Dev.              Interation with Demographic Variables            
                                                ————————————————————————————————————————————————————————
   Variable           β              σ           Income        IncomeSq          Age           Child     
————————————————————————————————————————————————————————————————————————————————————————————————————————
     Price         -62.730         3.313         588.324        -30.192                       11.055     
                  (14.803)        (1.34)        (270.44)       (14.101)                       (4.123)    
   Constant        -2.010          0.558          2.292                         1.284                    
                   (0.327)        (0.163)        (1.209)                       (0.631)                   
     Sugar          0.116         -0.006         